In [1]:
print('Link Damping Energy optimization')

Link Damping Energy optimization


In [2]:
import os
import sys
import comtypes.client
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pygad

In [3]:
SapObject = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
SapModel = SapObject.SapModel
SapModel.SetModelIsLocked(False)

0

In [14]:
def link_option(Y_strength, L_name):
    DOF = [False, False, False, False, False, False]
    Fixed = [False, False, False, False, False, False]
    NonLinear = [False, False, False, False, False, False]
    Ke = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Ce = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    K =[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Yield = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Ratio = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Exp =[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

    DOF[0] = True
    Fixed[0] = True

    DOF[1] = True
    Fixed[1] = True

    DOF[2] = True
    Fixed[2] = True

    DOF[3] = True
    Fixed[3] = True

    DOF[3] = True
    Fixed[3] = True

    DOF[4] = True
    Fixed[4] = True

    DOF[5] = True
    NonLinear[5] = True
    Ke[5] = 50000
    Ce[5] = 0
    K[5] = 50000
    Yield[5] = Y_strength
    Ratio[5] = 0.03
    Exp[5] = 2
    ret = SapModel.PropLink.SetPlasticWen(L_name, DOF, Fixed, NonLinear, Ke, Ce, K, Yield, Ratio, Exp, 2, 0)


def get_data(L_num, Plot_graph = False):
    eItemTypeElm = 1
    NumberResults = 0
    Obj =[]
    Elm =[]
    PointElm =[]
    LoadCase =[]
    StepType =[]
    StepNum =[]
    P =[]
    V2 =[]
    V3 =[]
    T =[]
    M2 =[]
    M3 =[]

    U1=[]
    U2=[]
    U3=[]
    R1=[]
    R2=[]
    R3=[]
    SapModel.Results.Setup.SetOptionModalHist(2)
    [NumberResults, Obj, Elm, LoadCase,PointElm, StepType, StepNum, P, V2, V3, T, M2, M3, ret] = SapModel.Results.LinkForce(L_num, eItemTypeElm, NumberResults, Obj, Elm, PointElm, LoadCase, StepType, StepNum, P, V2, V3, T, M2, M3)

    [NumberResults, Obj, Elm, LoadCase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret]= SapModel.Results.LinkDeformation(L_num, eItemTypeElm, NumberResults, Obj, Elm, LoadCase, StepType, StepNum, U1, U2, U3, R1, R2, R3)

    result = pd.DataFrame()
    result['Desplacment'] = R3
    result['Force'] = M3[0:len(M3):2]

    if Plot_graph:
        plt.plot(result['Desplacment'], result['Force'])
        plt.grid()
        plt.show()

    #Energy = np.abs(result['Desplacment'][len(result)-1]*result['Force'][0] - result['Desplacment'][0]*result['Force'][len(result)-1])/2
    Energy = 0
    for i in range(len(result)-1):
        #Area = result['Desplacment'][i]*result['Force'][i+1] - result['Desplacment'][i+1]*result['Force'][i]
        Area = (result['Force'][i] + result['Force'][i+1]) / 2 * (result['Desplacment'][i+1]- result['Desplacment'][i])
        Energy = Energy +  Area
    return result, np.abs(Energy)

In [15]:
Y_strength = [500, 100, 200, 100, 500]



def main(Y_strength):
    print(Y_strength)
    SapModel.SetModelIsLocked(False)
    Link_list = ['N-LIN1', 'LIN2','LIN3','LIN4','LIN5']
    #Link_numbers = ['1', '3', '5', '7', '9', '11', '13', '15', '17', '19']
    Link_numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']

    for link_name in Link_list:
        link_option( Y_strength[Link_list.index(link_name)], link_name)
    
    SapModel.Analyze.RunAnalysis()
    SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()
    ret = SapModel.Results.Setup.SetCaseSelectedForOutput("ACASE1")

    E = []
    for num in Link_numbers:
        r, e = get_data(num)
        E.append(e)
    print("E=",np.sum(E))
    print("1/E=",1/np.sum(E))

    return np.sum(E)
#23, 95

In [16]:
def fitness_func(solution, solution_idx):
    return 1/main(solution)

last_fitness = 0
def on_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1] - last_fitness))
    last_fitness = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]

fitness_function = fitness_func

num_generations = 10 #iteratinos
num_parents_mating = 2

sol_per_pop = 5 # solutions per iteration
num_genes = len(Y_strength)

init_range_low = 10
init_range_high = 300

parent_selection_type = "sss"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       on_generation=on_generation)

In [17]:
ga_instance.run()

[ 16.25164663 141.18631584  25.3938936   74.342362   292.76333591]
E= 0.2200517112155978
1/E= 4.544386382981773
[249.30606603 187.53479787 133.1984545   34.05842775 107.73519377]
E= 0.2890395246192998
1/E= 3.4597344474501246
[104.30253542 120.47094148 182.65202698  29.03273459 103.56051139]
E= 0.2999988978913905
1/E= 3.3333455790295368
[181.49430053 298.83923505 255.3181302  153.62558376 147.54711507]
E= 0.6162722421932393
1/E= 1.6226594864002304
[284.12997636 182.93576589 196.13651588  94.31653238 125.16921103]
E= 0.4771129413695875
1/E= 2.0959397938974935
[ 16.25164663 141.18631584 132.51975082  34.05842775 107.73519377]
E= 0.217887994868431
1/E= 4.589513986779481
[249.30606603 187.53479787 133.1984545   74.342362   293.4232821 ]
E= 0.34941833820472323
1/E= 2.8618990209211708
[249.10495501 187.53479787 133.1984545   34.05842775 107.73519377]
E= 0.28911820925089404
1/E= 3.4587928674261033
[249.30606603 187.53479787 133.51547987  74.342362   292.76333591]
E= 0.3501634763264843
1/E= 2.8